In [1]:
# The imports

# Define the test scorer
def competition_scorer(y_true, y_pred):
    return log_loss(y_true, y_pred, sample_weight=10**y_true)

import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss

In [2]:
# reading data using pandas

requests_train = requests_train_org = pd.read_csv('data/requests_train.csv', sep=',', low_memory=False, error_bad_lines=False)
requests_test = requests_test_org = pd.read_csv(filepath_or_buffer='data/requests_test.csv', sep=',', low_memory=False, error_bad_lines=False)

individuals_train = individuals_train_org = pd.read_csv(filepath_or_buffer='data/individuals_train.csv', sep=',', low_memory=False, error_bad_lines=False)
individuals_test = individuals_test_org = pd.read_csv(filepath_or_buffer='data/individuals_test.csv', sep=',', low_memory=False, error_bad_lines=False)

In [3]:
# data info request_train
requests_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238191 entries, 0 to 238190
Data columns (total 24 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   request_id                     238191 non-null  object
 1   animal_presence                238191 non-null  object
 2   answer_creation_date           238191 non-null  object
 3   child_situation                238191 non-null  int64 
 4   child_to_come                  92244 non-null   object
 5   district                       238191 non-null  int64 
 6   granted_number_of_nights       238191 non-null  int64 
 7   group_composition_id           238191 non-null  int64 
 8   group_composition_label        238191 non-null  object
 9   group_creation_date            238191 non-null  object
 10  group_id                       238191 non-null  object
 11  group_main_requester_id        238191 non-null  object
 12  group_type                     238191 non-nu

In [4]:
# data info request_test
requests_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59548 entries, 0 to 59547
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   request_id                     59548 non-null  object
 1   animal_presence                59548 non-null  object
 2   answer_creation_date           59548 non-null  object
 3   child_situation                59548 non-null  int64 
 4   child_to_come                  23209 non-null  object
 5   district                       59548 non-null  int64 
 6   granted_number_of_nights       59548 non-null  int64 
 7   group_composition_id           59548 non-null  int64 
 8   group_composition_label        59547 non-null  object
 9   group_creation_date            59547 non-null  object
 10  group_id                       59548 non-null  object
 11  group_main_requester_id        59548 non-null  object
 12  group_type                     59547 non-null  object
 13  h

In [5]:
# data info individuals_train
individuals_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384133 entries, 0 to 384132
Data columns (total 16 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   request_id                     384133 non-null  object
 1   individual_id                  384133 non-null  object
 2   housing_situation_2_id         384133 non-null  int64 
 3   housing_situation_2_label      384133 non-null  object
 4   birth_month                    384133 non-null  int64 
 5   birth_year                     384133 non-null  int64 
 6   childcare_center_supervision   5879 non-null    object
 7   disabled_worker_certification  129392 non-null  object
 8   gender                         384132 non-null  object
 9   individual_creation_date       364433 non-null  object
 10  individual_role                83090 non-null   object
 11  individual_role_2_id           384133 non-null  int64 
 12  individual_role_2_label        384078 non-nu

In [6]:
# data info individuals_test
individuals_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95641 entries, 0 to 95640
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   request_id                     95641 non-null  object
 1   individual_id                  95641 non-null  object
 2   housing_situation_2_id         95641 non-null  int64 
 3   housing_situation_2_label      95641 non-null  object
 4   birth_month                    95641 non-null  int64 
 5   birth_year                     95641 non-null  int64 
 6   childcare_center_supervision   1454 non-null   object
 7   disabled_worker_certification  32655 non-null  object
 8   gender                         95640 non-null  object
 9   individual_creation_date       90699 non-null  object
 10  individual_role                20703 non-null  object
 11  individual_role_2_id           95641 non-null  int64 
 12  individual_role_2_label        95628 non-null  object
 13  m

In [7]:
# looking at the data (requests_train)

pd.options.display.max_columns = None
requests_train.head(5)

,request_id,animal_presence,answer_creation_date,child_situation,child_to_come,district,granted_number_of_nights,group_composition_id,group_composition_label,group_creation_date,group_id,group_main_requester_id,group_type,housing_situation_id,housing_situation_label,long_term_housing_request,number_of_underage,request_backoffice_creator_id,request_creation_date,requester_type,social_situation_id,town,victim_of_violence,victim_of_violence_type
0,130667d2964de43c511d0ead7ac66b33,f,2019-02-11 22:30:00,-1,NaN,85,1,10,man alone,2018-05-03 12:10:40.416,c62f692dcc3f31880dd3937369c6f9e1,56252a8182a732cad0d106328bf96a0b,individual,160,other,NaN,-1,eecd6110b7e9157e6423428b22a28159,2019-02-11 22:30:00,third party,9a84febc47fedaf2c81d7c755e9edc85,NaN,f,NaN
1,b1cd62fcf24eebb0f685d5eaf55317a8,f,2019-02-12 09:00:00,-1,NaN,85,1,10,man alone,2019-03-07 12:11:08.545,cbb3ae0239a3ca9a0441831bde02aaa3,b23d21c9973e5b74504a94eebe82fb75,individual,160,other,NaN,-1,eecd6110b7e9157e6423428b22a28159,2019-02-12 09:00:00,user,11f64f0547e855c5c59329b9edc63063,NaN,f,NaN
2,12cc0a18890bd4959fe42df3ae58838e,f,2019-02-01 21:00:00,-1,NaN,59,1,80,isolated child/underage,2019-03-13 12:54:15.887,a73d85877e940fd36ac23bb80ac7fbbe,5e063c21a61a5b38f8f21baebea86f50,individual,200,street,NaN,-1,686d962128c7622aa79ed91da13f5c77,2019-02-01 21:00:00,user,898ab2883f410f4c6fb042ef16329fbb,NaN,f,NaN
3,ae2d5b4dc181d29e430132f145da1556,f,2019-02-25 15:12:05.037,-1,NaN,50,2,20,woman alone,2018-10-09 14:37:29.773,5f69b8b928f84ce1e673740acb854af2,f46e831e036fe418d88d6d40ee4b0680,individual,170,NaN,NaN,-1,ec29519464c381a6630f4e480a0c000e,2019-02-25 15:12:05.037,user,3998c8e890ddc980527dd1af45dfa672,NaN,t,woman
4,d13a17ce36c832514fda2464e11c4a9f,f,2019-02-20 09:59:20.603,-1,NaN,93,2,60,group with child(ren),2018-10-02 10:47:36.931,df3a2364091c2aa9f90f2738f6b176f1,936ae1e1737f38e8f9ef3049805629f3,group,50,hotel paid by the emergency centre,NaN,-1,c8b5cc926b1d7212224a08baa9619518,2019-02-20 09:59:20.603,user,74a10d96eae43586bc3c46e89719b821,NaN,f,NaN


In [8]:
# looking at the data (individuals)

individuals_train.head(5)

,request_id,individual_id,housing_situation_2_id,housing_situation_2_label,birth_month,birth_year,childcare_center_supervision,disabled_worker_certification,gender,individual_creation_date,individual_role,individual_role_2_id,individual_role_2_label,marital_status_id,marital_status_label,pregnancy
0,130667d2964de43c511d0ead7ac66b33,b24a8cfaaf3663bac45ca3bcd24f0b36,20,on the street,5,1977,NaN,NaN,male,2018-01-29,isolated person,30,isolated adult,10,single person,f
1,b1cd62fcf24eebb0f685d5eaf55317a8,eef783d1bda82661a751c12c34aff8c2,20,on the street,5,1980,NaN,NaN,male,2019-03-07,NaN,30,isolated adult,-1,NaN,f
2,12cc0a18890bd4959fe42df3ae58838e,927b6fa58d6be263c411332835b11fec,20,on the street,1,2003,NaN,NaN,male,2019-03-13,NaN,10,isolated underage/child,-1,NaN,f
3,ae2d5b4dc181d29e430132f145da1556,94181d5cf4a56cc84d12677df84e2e03,20,on the street,7,1986,NaN,NaN,female,2018-10-09,NaN,30,isolated adult,-1,NaN,f
4,d13a17ce36c832514fda2464e11c4a9f,e79d2e3f9ace50049f1796c098008c13,35,hotel paid by the emergency structure,9,1980,NaN,NaN,female,2017-06-22,husband/spouse,1,partner,-1,NaN,f


In [9]:
# merging the two data sets

# strategy 1: just merge them and ignore if we have duplicated in individuals
df_merged_train = pd.merge(left=requests_train, right=individuals_train, left_on='request_id', right_on='request_id')
df_merged_test = pd.merge(left=requests_test, right=individuals_test, left_on='request_id', right_on='request_id')

# strategy 2: delete the duplicated request_id in individuals and then merge
# individuals_train.drop_duplicates(subset='request_id', keep='first', inplace=True)
# individuals_test.drop_duplicates(subset='request_id', keep='first', inplace=True)

# df_merged_train.duplicated().any()
# df_merged_test.duplicated().any()

In [10]:
# looking at the data_merged (train)

df_merged_train.head(5)

,request_id,animal_presence,answer_creation_date,child_situation,child_to_come,district,granted_number_of_nights,group_composition_id,group_composition_label,group_creation_date,group_id,group_main_requester_id,group_type,housing_situation_id,housing_situation_label,long_term_housing_request,number_of_underage,request_backoffice_creator_id,request_creation_date,requester_type,social_situation_id,town,victim_of_violence,victim_of_violence_type,individual_id,housing_situation_2_id,housing_situation_2_label,birth_month,birth_year,childcare_center_supervision,disabled_worker_certification,gender,individual_creation_date,individual_role,individual_role_2_id,individual_role_2_label,marital_status_id,marital_status_label,pregnancy
0,130667d2964de43c511d0ead7ac66b33,f,2019-02-11 22:30:00,-1,NaN,85,1,10,man alone,2018-05-03 12:10:40.416,c62f692dcc3f31880dd3937369c6f9e1,56252a8182a732cad0d106328bf96a0b,individual,160,other,NaN,-1,eecd6110b7e9157e6423428b22a28159,2019-02-11 22:30:00,third party,9a84febc47fedaf2c81d7c755e9edc85,NaN,f,NaN,b24a8cfaaf3663bac45ca3bcd24f0b36,20,on the street,5,1977,NaN,NaN,male,2018-01-29,isolated person,30,isolated adult,10,single person,f
1,b1cd62fcf24eebb0f685d5eaf55317a8,f,2019-02-12 09:00:00,-1,NaN,85,1,10,man alone,2019-03-07 12:11:08.545,cbb3ae0239a3ca9a0441831bde02aaa3,b23d21c9973e5b74504a94eebe82fb75,individual,160,other,NaN,-1,eecd6110b7e9157e6423428b22a28159,2019-02-12 09:00:00,user,11f64f0547e855c5c59329b9edc63063,NaN,f,NaN,eef783d1bda82661a751c12c34aff8c2,20,on the street,5,1980,NaN,NaN,male,2019-03-07,NaN,30,isolated adult,-1,NaN,f
2,12cc0a18890bd4959fe42df3ae58838e,f,2019-02-01 21:00:00,-1,NaN,59,1,80,isolated child/underage,2019-03-13 12:54:15.887,a73d85877e940fd36ac23bb80ac7fbbe,5e063c21a61a5b38f8f21baebea86f50,individual,200,street,NaN,-1,686d962128c7622aa79ed91da13f5c77,2019-02-01 21:00:00,user,898ab2883f410f4c6fb042ef16329fbb,NaN,f,NaN,927b6fa58d6be263c411332835b11fec,20,on the street,1,2003,NaN,NaN,male,2019-03-13,NaN,10,isolated underage/child,-1,NaN,f
3,ae2d5b4dc181d29e430132f145da1556,f,2019-02-25 15:12:05.037,-1,NaN,50,2,20,woman alone,2018-10-09 14:37:29.773,5f69b8b928f84ce1e673740acb854af2,f46e831e036fe418d88d6d40ee4b0680,individual,170,NaN,NaN,-1,ec29519464c381a6630f4e480a0c000e,2019-02-25 15:12:05.037,user,3998c8e890ddc980527dd1af45dfa672,NaN,t,woman,94181d5cf4a56cc84d12677df84e2e03,20,on the street,7,1986,NaN,NaN,female,2018-10-09,NaN,30,isolated adult,-1,NaN,f
4,d13a17ce36c832514fda2464e11c4a9f,f,2019-02-20 09:59:20.603,-1,NaN,93,2,60,group with child(ren),2018-10-02 10:47:36.931,df3a2364091c2aa9f90f2738f6b176f1,936ae1e1737f38e8f9ef3049805629f3,group,50,hotel paid by the emergency centre,NaN,-1,c8b5cc926b1d7212224a08baa9619518,2019-02-20 09:59:20.603,user,74a10d96eae43586bc3c46e89719b821,NaN,f,NaN,e79d2e3f9ace50049f1796c098008c13,35,hotel paid by the emergency structure,9,1980,NaN,NaN,female,2017-06-22,husband/spouse,1,partner,-1,NaN,f


In [11]:
# some information about the data train

df_merged_train.shape
# df_merged_train.info()
# df_merged_train.describe()

(384133, 39)

In [12]:
# some information about the data test

df_merged_test.shape
# df_merged_test.info()
# df_merged_test.describe()

(95641, 39)

In [13]:
# number of NaN of each column in data train
nan_coulmn = df_merged_train.isnull().sum(axis=0)
nan_coulmn

request_id                            0
animal_presence                       0
answer_creation_date                  0
child_situation                       0
child_to_come                    232192
district                              0
granted_number_of_nights              0
group_composition_id                  0
group_composition_label               0
group_creation_date                   0
group_id                              0
group_main_requester_id               0
group_type                            0
housing_situation_id                  0
housing_situation_label           23309
long_term_housing_request        263872
number_of_underage                    0
request_backoffice_creator_id         0
request_creation_date                 0
requester_type                        0
social_situation_id                   0
town                             253287
victim_of_violence                    0
victim_of_violence_type          377088
individual_id                         0


In [14]:
# impute the nan values (if possible)
# 'gender' and 'pregnancy' have small amount of nan, so just remove that rows

df_merged_train = df_merged_train.dropna(subset=['gender','pregnancy'])
nan_coulmn = df_merged_train.isnull().sum(axis=0)
nan_coulmn

request_id                            0
animal_presence                       0
answer_creation_date                  0
child_situation                       0
child_to_come                    232184
district                              0
granted_number_of_nights              0
group_composition_id                  0
group_composition_label               0
group_creation_date                   0
group_id                              0
group_main_requester_id               0
group_type                            0
housing_situation_id                  0
housing_situation_label           23309
long_term_housing_request        263863
number_of_underage                    0
request_backoffice_creator_id         0
request_creation_date                 0
requester_type                        0
social_situation_id                   0
town                             253279
victim_of_violence                    0
victim_of_violence_type          377073
individual_id                         0


In [15]:
# number of NaN in data test
nan_coulmn_test = df_merged_test.isnull().sum(axis=0)
nan_coulmn_test

request_id                           0
animal_presence                      0
answer_creation_date                 0
child_situation                      0
child_to_come                    57361
district                             0
granted_number_of_nights             0
group_composition_id                 0
group_composition_label              1
group_creation_date                  1
group_id                             0
group_main_requester_id              0
group_type                           1
housing_situation_id                 0
housing_situation_label           5858
long_term_housing_request        65347
number_of_underage                   0
request_backoffice_creator_id        0
request_creation_date                0
requester_type                       0
social_situation_id                  0
town                             62547
victim_of_violence                   1
victim_of_violence_type          93988
individual_id                        0
housing_situation_2_id   

In [16]:
# handle the missing value in data test
# strategy 1: drop it
df_merged_test = df_merged_test.dropna(subset=['gender','pregnancy'])
nan_coulmn_test = df_merged_test.isnull().sum(axis=0)
nan_coulmn_test

# strategy 2: impute it (using mode)
# df_merged_test.mode()
# df_merged_test[df_merged_test['group_composition_label'].isnull()] = df_merged_test.mode()['group_composition_label']
# df_merged_test[df_merged_test['group_creation_date'].isnull()] = df_merged_test.mode()['group_creation_date']
# df_merged_test[df_merged_test['group_type'].isnull()] = df_merged_test.mode()['group_type']
# df_merged_test[df_merged_test['victim_of_violence'].isnull()] = df_merged_test.mode()['victim_of_violence']
# df_merged_test[df_merged_test['gender'].isnull()] = df_merged_test.mode()['gender']
# df_merged_test[df_merged_test['pregnancy'].isnull()] = df_merged_test.mode()['pregnancy']
# ignore 'individual_role_2_label' because we will use the 'marital_status_id' which is the same

request_id                           0
animal_presence                      0
answer_creation_date                 0
child_situation                      0
child_to_come                    57357
district                             0
granted_number_of_nights             0
group_composition_id                 0
group_composition_label              0
group_creation_date                  0
group_id                             0
group_main_requester_id              0
group_type                           0
housing_situation_id                 0
housing_situation_label           5858
long_term_housing_request        65343
number_of_underage                   0
request_backoffice_creator_id        0
request_creation_date                0
requester_type                       0
social_situation_id                  0
town                             62543
victim_of_violence                   0
victim_of_violence_type          93984
individual_id                        0
housing_situation_2_id   

In [17]:
# drop nan coulmns (the columns with huge amounts of nan)

df_merged_train = df_merged_train.loc[:, nan_coulmn==0]
df_merged_test = df_merged_test.loc[:, nan_coulmn==0]

In [18]:
# checking the number of variables in each column train

df_merged_train.nunique()

request_id                       238186
animal_presence                       2
answer_creation_date             222751
child_situation                       8
district                            102
granted_number_of_nights              4
group_composition_id                 12
group_composition_label              12
group_creation_date               55644
group_id                          56850
group_main_requester_id           54996
group_type                            2
housing_situation_id                 22
number_of_underage                   13
request_backoffice_creator_id      1069
request_creation_date            222879
requester_type                        3
social_situation_id               56822
victim_of_violence                    2
individual_id                     83504
housing_situation_2_id               28
housing_situation_2_label            28
birth_month                          12
birth_year                          113
gender                                2


In [19]:
# selecting the features
train_data = df_merged_train.copy()
test_data = df_merged_test.copy()

train_data = train_data.iloc[:, [1, 3, 4, 5, 7, 11, 12, 13, 16, 18, 20, 24, 25, 26, 27]]
test_data = test_data.iloc[:, [1, 3, 4, 5, 7, 11, 12, 13, 16, 18, 20, 24, 25, 26, 27]]

# train_data = train_data.iloc[:, [3, 4, 5, 7, 12, 13, 20, 25, 26]]
# test_data = test_data.iloc[:, [3, 4, 5, 7, 12, 13, 20, 25, 26]]

In [20]:
# correlation matrix
corr = df_merged_train.corr()
corr.style.background_gradient(cmap='coolwarm')

,child_situation,district,granted_number_of_nights,group_composition_id,housing_situation_id,number_of_underage,housing_situation_2_id,birth_month,birth_year,individual_role_2_id,marital_status_id
child_situation,1.000000,-0.031477,0.015255,0.131072,-0.006054,0.374624,0.071410,0.005846,0.062863,0.089382,0.051877
district,-0.031477,1.000000,-0.008614,-0.012618,0.054437,-0.050374,-0.049537,-0.010152,-0.002395,-0.019250,-0.021679
granted_number_of_nights,0.015255,-0.008614,1.000000,0.024349,-0.458577,0.039587,0.283849,0.008160,0.015521,0.052489,-0.006058
group_composition_id,0.131072,-0.012618,0.024349,1.000000,-0.022739,0.240794,0.137080,0.036148,0.229676,0.208428,0.177774
housing_situation_id,-0.006054,0.054437,-0.458577,-0.022739,1.000000,-0.020491,-0.307071,-0.005274,0.014052,-0.051637,-0.003608
number_of_underage,0.374624,-0.050374,0.039587,0.240794,-0.020491,1.000000,0.081769,0.013291,0.108102,0.158572,0.023907
housing_situation_2_id,0.071410,-0.049537,0.283849,0.137080,-0.307071,0.081769,1.000000,0.009535,0.073313,0.097054,0.014914
birth_month,0.005846,-0.010152,0.008160,0.036148,-0.005274,0.013291,0.009535,1.000000,0.011294,0.029515,0.000392
birth_year,0.062863,-0.002395,0.015521,0.229676,0.014052,0.108102,0.073313,0.011294,1.000000,0.329884,-0.106394
individual_role_2_id,0.089382,-0.019250,0.052489,0.208428,-0.051637,0.158572,0.097054,0.029515,0.329884,1.000000,-0.389138


In [21]:
# Converting string columns in data

le = preprocessing.LabelEncoder()
# transforming columns in train and test data
for column in train_data.columns:
    le.fit(train_data[column])
    train_data[column] = le.transform(train_data[column])
    test_data[column] = le.transform(test_data[column])


In [22]:
# correlation matrix
corr = train_data.corr()
corr.style.background_gradient(cmap='coolwarm')

,animal_presence,child_situation,district,granted_number_of_nights,group_composition_label,group_type,housing_situation_id,number_of_underage,requester_type,victim_of_violence,housing_situation_2_id,gender,individual_role_2_id,marital_status_id,pregnancy
animal_presence,1.000000,-0.008558,-0.020011,-0.004991,0.007890,0.026616,-0.007767,-0.011451,-0.007244,0.000973,0.000157,0.014354,-0.023600,0.005976,-0.007380
child_situation,-0.008558,1.000000,0.068691,0.014486,-0.030468,-0.148742,-0.006062,0.363383,0.011938,0.047498,0.069283,-0.072348,0.084861,0.050592,-0.006401
district,-0.020011,0.068691,1.000000,0.067154,0.006967,-0.119496,0.050824,0.132931,0.010907,0.001992,0.094237,-0.088708,0.014798,0.048514,0.052447
granted_number_of_nights,-0.004991,0.014486,0.067154,1.000000,0.045883,-0.035855,-0.458577,0.039603,0.010342,0.039192,0.283849,-0.049905,0.052246,-0.005214,-0.009873
group_composition_label,0.007890,-0.030468,0.006967,0.045883,1.000000,0.506327,-0.031665,-0.123406,-0.008134,0.145335,-0.022595,-0.084336,0.033284,-0.270230,0.001865
group_type,0.026616,-0.148742,-0.119496,-0.035855,0.506327,1.000000,0.011281,-0.235121,-0.021251,-0.036511,-0.171310,0.322466,-0.271929,-0.180771,-0.044213
housing_situation_id,-0.007767,-0.006062,0.050824,-0.458577,-0.031665,0.011281,1.000000,-0.020506,-0.018597,-0.041022,-0.307071,0.035136,-0.051614,-0.003973,0.019566
number_of_underage,-0.011451,0.363383,0.132931,0.039603,-0.123406,-0.235121,-0.020506,1.000000,0.007004,0.044573,0.081786,-0.078343,0.156775,0.023751,-0.021012
requester_type,-0.007244,0.011938,0.010907,0.010342,-0.008134,-0.021251,-0.018597,0.007004,1.000000,-0.023291,-0.033656,-0.007432,0.009993,0.008960,0.005336
victim_of_violence,0.000973,0.047498,0.001992,0.039192,0.145335,-0.036511,-0.041022,0.044573,-0.023291,1.000000,0.087399,-0.098083,0.058269,0.010455,0.007167


In [33]:
### PCA

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

features = train_data.columns

# dropping the target column from features
features.drop('granted_number_of_nights')

# Separating out the features
x_train = train_data.loc[:, features].values
x_test = test_data.loc[:, features].values

# Separating out the target
y_train = train_data.loc[:,['granted_number_of_nights']].values
y_test = test_data.loc[:,['granted_number_of_nights']].values

# Standardizing the features
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

# PCA
pca = PCA(n_components=8)

principalComponents = pca.fit_transform(x_train)
principalDf_train = pd.DataFrame(data = principalComponents)

principalDf_test = pd.DataFrame(data = principalComponents)

In [32]:
# correlation matrix
corr = pd.DataFrame(principalDf_train).corr()
corr.style.background_gradient(cmap='coolwarm')

,0,1,2,3,4,5,6,7
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000
1,0.000000,1.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000
2,0.000000,-0.000000,1.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,1.000000,-0.000000,0.000000,-0.000000,-0.000000
4,0.000000,-0.000000,-0.000000,-0.000000,1.000000,-0.000000,0.000000,0.000000
5,0.000000,0.000000,-0.000000,0.000000,-0.000000,1.000000,0.000000,0.000000
6,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,1.000000,0.000000
7,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,1.000000


In [23]:
# making x_train, x_test, y_train and y_test

# the outputs
y_train = train_data['granted_number_of_nights'].values
y_test = test_data['granted_number_of_nights'].values

# drop the outputs from data
train_data.drop(columns=['granted_number_of_nights'], inplace=True)
test_data.drop(columns=['granted_number_of_nights'], inplace=True)

# the inputs
x_train = train_data.values
x_test = test_data.values

In [23]:
# Evaluating some algorithms
# This block takes time, so you can just ignore this block of code

# Buliding models to find the best one
models = []
models.append(('LR', LogisticRegression(solver='liblinear', class_weight={0: 1, 1: 10, 2: 100, 3: 1000})))
models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(max_depth=10, class_weight={0: 1, 1: 10, 2: 100, 3: 1000})))
models.append(('NB', GaussianNB()))
models.append(('R-Foreset', RandomForestClassifier(n_estimators=10, max_depth=10, class_weight={0: 1, 1: 10, 2: 100, 3: 1000})))
# models.append(('SVM', SVC(gamma='auto', class_weight={0: 1, 1: 10, 2: 100, 3: 1000})))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring='neg_log_loss')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -1.443486 (0.001510)
LDA: -0.891517 (0.004099)
CART: -1.971295 (0.012780)
NB: -1.432458 (0.014836)
R-Foreset: -2.205472 (0.023986)


In [24]:
# Make predictions on validation dataset
# model = LogisticRegression(solver='newton-cg', max_iter=100, class_weight={0: 1, 1: 10, 2: 100, 3: 1000})
model = RandomForestClassifier(n_estimators=40, min_samples_split=10, max_depth=20, class_weight={0: 1, 1: 10, 2: 100, 3: 1000})
# lots of models are tried. This model reduces the accuracy but has less penalty for higher classes.
# So, in general, it performs good for this task, just like what is asked!

model.fit(x_train, y_train)
predictions = model.predict_proba(x_test)

In [25]:
# Evaluate predictions (using model.predict)
# predictions2 = model.predict(x_test)
# y_pred = [(predictions2==0)*1, (predictions2==1)*1, (predictions2==2)*1, (predictions2==3)*1]
# y_pred = np.array(y_pred)
# y_pred = y_pred.transpose()

# # Evaluate predictions based on what is asked in the test
# score = competition_scorer(y_test, y_pred)

# print(f'test score: {score}')
# print(accuracy_score(y_test, predictions2))
# print(confusion_matrix(y_test, predictions2))
# print(classification_report(y_test, predictions2))

In [26]:
# Evaluate predictions based on what is asked in the test (using model.predict_proba)
score = competition_scorer(y_test, predictions)

print(f'test score: {score}')

test score: 0.6847792180989747
